This android manifest is downloaded from the android source code at: https://cs.android.com/android/platform/superproject/+/master:frameworks/base/core/res/AndroidManifest.xml;l=3?q=AndroidManifest.xml&ss=android%2Fplatform%2Fsuperproject

Downloaded 17/12/2021

In this document we parse the runtime permissions given in the document, and print the content in a latex tabular format

In [1]:
import os

In [2]:
def load_manifest():
    manifest = os.getcwd() + '\\AndroidManifest.xml'
    return open(manifest, 'r', encoding='UTF-8')

In [3]:
class PermissionGroup:
    def __init__(self):
        self.name = None
        self.icon = None
        self.label = None
        self.description = None
        self.priority = None
        self.comment = None

In [4]:
class Permission:
    def __init__(self):
        self.name = None
        self.permissionGroup = None 
        self.backgroundPermission = None
        self.label = None 
        self.dscription = None 
        self.permissionFlags = None 
        self.protectionLevel = None
        self.isPartOf = None
        self.comment = None

In [5]:
def get_runtime_segment(lines):
    start = lines.index("<!--                          RUNTIME PERMISSIONS                           -->")
    end = lines.index("<!-- REMOVED PERMISSIONS                                                    -->")
    return lines[start:end]

In [6]:
def next_element(comment=None):
    i, l = next(segment_line)
    if (l.startswith('<!-- ===')): return comment # special case
    if (l.startswith(_is['start_comment'])): return parse_comment(l)
    if (l.startswith(_is['start_permission/group'])): 
        if (l.startswith(_is['group'])): return parse_group(l, comment, PermissionGroup())
        return parse_permission(l, comment, Permission())

In [7]:
# return a string which is a xml comment 
def parse_comment(l, comment=''):
    if (l.startswith(_is['start_comment'])): l = l.lstrip('<!-- ')
    comment += l
    if (l.endswith(_is['end_comment'])): return comment.rstrip(' -->')
    i, l = next(segment_line)
    return parse_comment(l, comment + ' ')

In [8]:
# return tuple of group and last parsed comment
def parse_group(l, comment, group):
    # extraction
    param = l[l.index(":")+1:l.index("=")]
    value = l[l.index("\"")+1:l.rindex("\"")]
    
    # assignment
    if param == 'name': group.name = value
    if param == 'icon': group.icon = value
    if param == 'label': group.label = value
    if param == 'description': group.description = value
    if param == 'priority': group.priority = value
    
    if (l.endswith(_is['end_permission/group'])): 
        group.comment = comment
        return group
    
    i, l = next(segment_line)
    return parse_group(l, comment, group)

In [9]:
# return tuple of permission and last parsed comment
def parse_permission(l, comment, permission):
    # extraction
    param = l[l.index(":")+1:l.index("=")]
    value = l[l.index("\"")+1:l.rindex("\"")]
    
    # assignment
    if param == 'name': permission.name = value
    if param == 'permissionGroup': permission.permission_group = value
    if param == 'label': permission.label = value
    if param == 'description': permission.description = value
    if param == 'permissionFlags': permission.permissionFlags = value
    if param == 'protectionLevel': permission.protectionLevel = value
    if param == 'backgroundPermission': permission.backgroundPermission = value
    
    if (l.endswith(_is['end_permission/group'])): 
        permission.comment = comment
        return permission
    
    i, l = next(segment_line)
    return parse_permission(l, comment, permission)

#### Parsing the document

In [10]:
# targets for our xml parsing
_is = {'start_comment': '<!--', 
       'end_comment': '-->', 
       'start_permission/group': 
       '<permission', 
       'end_permission/group': "/>", 
       'group':'<permission-group'}

In [11]:
manifest = load_manifest()
manifest_lines = [l.strip() for l in manifest]
runetime_segment = get_runtime_segment(manifest_lines)
segment_line = enumerate(runetime_segment)

groups = []
permissions = []
comment = ''
while(True):
    try:
        element = next_element(comment)
        if isinstance(element, PermissionGroup): groups.append(element)
        if isinstance(element, str): comment = element
        if isinstance(element, Permission): 
            element.isPartOf = groups[-1]
            permissions.append(element)
    except StopIteration: break

#### Writing group in the format of latex table content

In [12]:
print('Groups:', len(groups), '|', 'Permissions:', len(permissions))

Groups: 13 | Permissions: 50


In [13]:
i = 0
for p in permissions:
    #keep suffix after .
    name = p.name[p.name.rindex(".")+1:]
    isPartOf = p.isPartOf.name[p.isPartOf.name.rindex(".")+1:]
    backgroundPermission = p.backgroundPermission[p.backgroundPermission.rindex(".")+1:] if p.backgroundPermission else None 
    #write
    #protectionLevels = p.protectionLevel.split('|') # only display the first
    if '@hide' in p.comment: continue
    i += 1 # for enumeration of the permissions
    print('{} & {} & {} & {}\\\\'.format(i, name.replace('_', '\\_'), p.protectionLevel.replace('|', '$|$'), isPartOf.replace('_', '\\_')))

1 & READ\_CONTACTS & dangerous & CONTACTS\\
2 & WRITE\_CONTACTS & dangerous & CONTACTS\\
3 & READ\_CALENDAR & dangerous & CALENDAR\\
4 & WRITE\_CALENDAR & dangerous & CALENDAR\\
5 & SEND\_SMS & dangerous & SMS\\
6 & RECEIVE\_SMS & dangerous & SMS\\
7 & READ\_SMS & dangerous & SMS\\
8 & RECEIVE\_WAP\_PUSH & dangerous & SMS\\
9 & RECEIVE\_MMS & dangerous & SMS\\
10 & READ\_EXTERNAL\_STORAGE & dangerous & STORAGE\\
11 & WRITE\_EXTERNAL\_STORAGE & dangerous & STORAGE\\
12 & ACCESS\_MEDIA\_LOCATION & dangerous & STORAGE\\
13 & MANAGE\_EXTERNAL\_STORAGE & signature$|$appop$|$preinstalled & STORAGE\\
14 & MANAGE\_MEDIA & signature$|$appop$|$preinstalled & STORAGE\\
15 & ACCESS\_FINE\_LOCATION & dangerous$|$instant & LOCATION\\
16 & ACCESS\_COARSE\_LOCATION & dangerous$|$instant & LOCATION\\
17 & ACCESS\_BACKGROUND\_LOCATION & dangerous$|$instant & LOCATION\\
18 & READ\_CALL\_LOG & dangerous & CALL\_LOG\\
19 & WRITE\_CALL\_LOG & dangerous & CALL\_LOG\\
20 & PROCESS\_OUTGOING\_CALLS & dangerous